In [1]:
# Build a Resnet model from scratch
import os  # OS related functions
import numpy as np  # Numerical functions
import pandas as pd  # Data manipulation
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Plotting

# Deep learning
import tensorflow as tf
from keras.models import Sequential  # Pipeline
from keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Callbacks
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,
)  # Image data generator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Add

# Sklearn
from sklearn.metrics import classification_report  # Metrics
from sklearn.utils.class_weight import compute_class_weight  # Class weights
from sklearn.model_selection import train_test_split

print(
    "GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE"
)

2024-05-16 16:20:33.851408: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 16:20:33.851452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 16:20:33.852325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-16 16:20:33.857816: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 16:20:34.656817: W tensorflow/compiler/tf2

GPU is available


2024-05-16 16:20:36.538405: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-16 16:20:36.577253: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-16 16:20:36.577526: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
input_shape = (256, 256, 3)
image_size = 256
batch_size = 32
num_classes = 17

df_train = pd.read_csv("../../data/processed/train.csv")  # Load train data
df_test = pd.read_csv("../../data/test/test.csv")  # Load test data

# convert \ to / in path
df_train["Path"] = df_train["Path"].str.replace("\\", "/")
df_test["Path"] = df_test["Path"].str.replace("\\", "/")

df_train.info()  # Display data information
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196626 entries, 0 to 196625
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Path      196626 non-null  object 
 1   Category  196626 non-null  object 
 2   Style     196626 non-null  object 
 3   Width     196626 non-null  int64  
 4   Height    196626 non-null  int64  
 5   MinValue  196626 non-null  int64  
 6   MaxValue  196626 non-null  int64  
 7   StdDev    196626 non-null  float64
dtypes: float64(1), int64(4), object(3)
memory usage: 12.0+ MB


,Path,Category,Style,Width,Height,MinValue,MaxValue,StdDev
0,../../data/raw/Furniture_Data/Furniture_Data/d...,dressers,Asian,256,256,0,255,78.264032
1,../../data/raw/Furniture_Data/Furniture_Data/l...,lamps,Asian,256,256,0,255,56.407194
2,../../data/raw/Furniture_Data/Furniture_Data/d...,dressers,Asian,256,256,0,255,69.652896
3,../../data/raw/Furniture_Data/Furniture_Data/l...,lamps,Asian,256,256,5,255,84.765229
4,../../data/raw/Furniture_Data/Furniture_Data/l...,lamps,Asian,256,256,2,255,56.764965


In [3]:
num_classes = df_train["Style"].nunique()  # Number of classes

# Image data generator
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Separate the test dataset for validation
df_val, df_test = train_test_split(df_test, test_size=0.3, random_state=42)

batch_size = 32  # Number of samples per gradient update
num_classes = df_train["Style"].nunique()  # Number of classes

# Image data generator
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Common arguments
common_args = {
    "x_col": "Path",  # Path to image
    "y_col": "Style",  # Target column
    "batch_size": batch_size,  # Batch size
    "class_mode": "categorical",  # Multi-class classification
    "target_size": (image_size, image_size),  # Specify the target image size
}

# Create generator for training data
train_dataset = datagen.flow_from_dataframe(
    dataframe=df_train,  # Training data
    shuffle=True,  # Shuffle the data
    **common_args  # Common arguments
)

# Create generator for testing data
val_dataset = datagen.flow_from_dataframe(
    dataframe=df_val,  # Testing data
    shuffle=False,  # Do not shuffle the data
    **common_args  # Common arguments
)

Found 196626 validated image filenames belonging to 17 classes.
Found 11475 validated image filenames belonging to 17 classes.


In [4]:
labels = {value: key for key, value in train_dataset.class_indices.items()}

print("Label Mappings for different categories:")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for different categories:
0 : Asian
1 : Beach
2 : Contemporary
3 : Craftsman
4 : Eclectic
5 : Farmhouse
6 : Industrial
7 : Mediterranean
8 : Midcentury
9 : Modern
10 : Rustic
11 : Scandinavian
12 : Southwestern
13 : Traditional
14 : Transitional
15 : Tropical
16 : Victorian


Link for the overview of the building block of Resnet: https://pytorch.org/hub/pytorch_vision_resnet/

In [5]:
# Implement Resnet34 from scratch
class ConvLayer(models.Model):
    def __init__(self, filters, kernel_size, strides=(1, 1)):
        super(ConvLayer, self).__init__(name="conv_layer")

        self.conv = layers.Conv2D(
            filters, kernel_size, strides=strides, padding="same", activation="relu"
        )
        self.bn = layers.BatchNormalization()

    def call(self, inputs, training=False):
        x = self.conv(inputs)
        x = self.bn(x, training=training)

        return x


class ResnetBlock(models.Model):
    def __init__(self, filters, strides=(1, 1)):
        super(ResnetBlock, self).__init__(name="resnet_block")

        self.conv1 = ConvLayer(filters, (3, 3), strides=strides)
        self.conv2 = ConvLayer(filters, (3, 3))
        self.activation = layers.Activation("relu")

        if strides != (1, 1):
            self.conv3 = ConvLayer(filters, (1, 1), strides=strides)

    def call(self, inputs, training=False):
        x = self.conv1(inputs, training=training)
        x = self.conv2(x, training=training)

        if hasattr(self, "conv3"):
            x_add = self.conv3(inputs, training=training)
            x_add = Add()([x, x_add])
        else:
            x_add = Add()([x, inputs])

        return self.activation(x_add)


class Resnet34(models.Model):
    def __init__(self, num_classes):
        super(Resnet34, self).__init__(name="resnet34")

        self.conv1 = ConvLayer(64, (7, 7), strides=(2, 2))
        self.maxpool = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same")

        # Residual blocks
        self.conv2_1 = ResnetBlock(64)
        self.conv2_2 = ResnetBlock(64)
        self.conv2_3 = ResnetBlock(64)

        self.conv3_1 = ResnetBlock(128, strides=(2, 2))
        self.conv3_2 = ResnetBlock(128)
        self.conv3_3 = ResnetBlock(128)
        self.conv3_4 = ResnetBlock(128)

        self.conv4_1 = ResnetBlock(256, strides=(2, 2))
        self.conv4_2 = ResnetBlock(256)
        self.conv4_3 = ResnetBlock(256)
        self.conv4_4 = ResnetBlock(256)
        self.conv4_5 = ResnetBlock(256)
        self.conv4_6 = ResnetBlock(256)

        self.conv5_1 = ResnetBlock(512, strides=(2, 2))
        self.conv5_2 = ResnetBlock(512)
        self.conv5_3 = ResnetBlock(512)

        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.maxpool(x)

        x = self.conv2_1(x, training=True)
        x = self.conv2_2(x, training=True)
        x = self.conv2_3(x, training=True)

        x = self.conv3_1(x, training=True)
        x = self.conv3_2(x, training=True)
        x = self.conv3_3(x, training=True)
        x = self.conv3_4(x, training=True)

        x = self.conv4_1(x, training=True)
        x = self.conv4_2(x, training=True)
        x = self.conv4_3(x, training=True)
        x = self.conv4_4(x, training=True)
        x = self.conv4_5(x, training=True)
        x = self.conv4_6(x, training=True)

        x = self.conv5_1(x, training=True)
        x = self.conv5_2(x, training=True)
        x = self.conv5_3(x, training=True)

        x = self.avgpool(x)

        return self.fc(x)

In [6]:
resnet_34 = Resnet34(num_classes)
resnet_34(tf.zeros((1, 256, 256, 3)), training=False)

resnet_34.summary()

2024-05-16 16:20:37.772135: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-16 16:20:37.772461: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-16 16:20:37.772653: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-16 16:20:37.881836: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-16 16:20:37.882222: I external/local_xla/xla/stream_executor

Model: "resnet34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_layer (ConvLayer)      multiple                  9728      
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
 resnet_block (ResnetBlock)  multiple                  74368     
                                                                 
 resnet_block (ResnetBlock)  multiple                  74368     
                                                                 
 resnet_block (ResnetBlock)  multiple                  74368     
                                                                 
 resnet_block (ResnetBlock)  multiple                  231296    
                                                          

# Model Training and Inference

In [7]:
# Early stopping
early_stopping = EarlyStopping(
    monitor="val_loss",  # Monitor validation loss
    patience=5,  # Stop training if no improvement for 5 epochs
    restore_best_weights=True,  # Restore the best weights when stopping
    min_delta=0.001,  # Minimum change to qualify as an improvement
    verbose=1,  # Print messages
)

# Reduce learning rate
reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy",  # Monitor validation loss
    patience=5,  # Reduce learning rate if no improvement for 3 epochs
    factor=0.5,  # Factor by 0.5
    min_lr=0.00001,  # Minimum learning rate
    verbose=1,  # Print messages
)

# Calculate class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(df_train["Style"]),
    y=df_train["Style"],
)

optimizer = Adam(learning_rate=0.0001)

# Convert class weights to dictionary
class_weight_dict = dict(enumerate(class_weights))

# Compile the model
resnet_34.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)  # Compile model

# CSV logger
csv_logger = tf.keras.callbacks.CSVLogger(
    "./cache_cnn/training.csv", separator=",", append=False
)  # CSV logger

In [10]:
resnet_34.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32,
    callbacks=[early_stopping, reduce_lr, csv_logger],
    class_weight=class_weight_dict,
)

Epoch 1/32
6145/6145 [==============================] - 2258s 367ms/step - loss: 0.9255 - accuracy: 0.4658 - val_loss: 1.9601 - val_accuracy: 0.3800 - lr: 1.0000e-04
Epoch 2/32
6145/6145 [==============================] - 2257s 367ms/step - loss: 0.8219 - accuracy: 0.4934 - val_loss: 1.9940 - val_accuracy: 0.3848 - lr: 1.0000e-04
Epoch 3/32
6145/6145 [==============================] - 2257s 367ms/step - loss: 0.7412 - accuracy: 0.5206 - val_loss: 2.0404 - val_accuracy: 0.3866 - lr: 1.0000e-04
Epoch 4/32
6145/6145 [==============================] - 2258s 367ms/step - loss: 0.6597 - accuracy: 0.5499 - val_loss: 2.0658 - val_accuracy: 0.3956 - lr: 1.0000e-04
Epoch 5/32
6145/6145 [==============================] - 2258s 367ms/step - loss: 0.6080 - accuracy: 0.5720 - val_loss: 2.1224 - val_accuracy: 0.3929 - lr: 1.0000e-04
Epoch 6/32
6145/6145 [==============================] - 2257s 367ms/step - loss: 0.5574 - accuracy: 0.5948 - val_loss: 2.1005 - val_accuracy: 0.4037 - lr: 1.0000e-04
Epoc

In [9]:
# Save the model
resnet_34.save("./cache_resnet/r34.h5")  # Save model
print(f"Model saved to ./cache_resnet/r34.h5")

# Save the weights
resnet_34.save_weights("./cache_resnet/weights_34.h5")  # Save weights
print(f"Weights saved to ./cache_resnet/weights_34.h5")

# Save model
resnet_34.save("./cache_resnet/r34")

/home/mudoker/anaconda3/envs/py31/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.